***Challenge 1***

Here the goal is to train on 25 samples. In this preliminary testbed the evaluation will be done on a 2000 sample validation set. Note in the end the final evaluation will be done on the full CIFAR-10 test set as well as potentially a separate dataset. The validation samples here should not be used for training in any way, the final evaluation will provide only random samples of 25 from a datasource that is not the CIFAR-10 training data. 

Feel free to modify this testbed to your liking, including the normalization transformations etc. Note however the final evaluation testbed will have a rigid set of components where you will need to place your answer. The only constraint is the data. Refer to the full project instructions for more information.


Setup training functions. Again you are free to fully modify this testbed in your prototyping within the constraints of the data used. You can use tools outside of pytorch for training models if desired as well although the torchvision dataloaders will still be useful for interacting with the cifar-10 dataset. 

In [177]:
def train(model, device, train_loader, optimizer, epoch, display=True):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
    if display:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

In [179]:
import torch
import torch.nn as nn 
import torch.nn.functional as F

class SmallNet(nn.Module):
    def __init__(self):
        super(SmallNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 8 * 8, 1024)
        self.drop = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(1024,10)

    def forward(self, x):
        #x = x.unsqueeze(0)
        x = self.pool1(nn.functional.relu(self.bn1(self.conv1(x))))
        x = nn.functional.relu(self.conv2(x))
        x = self.pool2(nn.functional.relu(self.bn2(self.conv3(x))))
        x = x.view(-1, 32 * 8 * 8)
        x = nn.functional.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return torch.nn.functional.sigmoid(x)

In [180]:
import math
import numpy as np
from PIL import Image

class RotationTransform:
    def __init__(self, angle):
        self.angle = angle

    def __call__(self, x):
        angle = math.radians(self.angle)
        cos_theta = math.cos(angle)
        sin_theta = math.sin(angle)
        width, height = x.size
        cx = width / 2
        cy = height / 2
        x_transformed = x.rotate(self.angle, resample=Image.BICUBIC)
        x_transformed = transforms.functional.crop(x_transformed, cx - 16, cy - 16, 32, 32)
        return x_transformed

The below tries  2 random problem instances. In your development you may choose to prototype with 1 problem instances but keep in mind for small sample problems the variance is high so continously evaluating on several subsets will be important.

In [227]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset

  
from torchvision import datasets, transforms
normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))

transform_val = transforms.Compose([transforms.ToTensor(), normalize]) #careful to keep this one same
transform_train = transforms.Compose([transforms.ToTensor(), normalize]) 

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# define transformation pipeline
transform_train_rotate1 = transforms.Compose([
    # transforms.ToPILImage(),
    RotationTransform(-30),
    # RotationTransform(30),
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                        std=[0.2023, 0.1994, 0.2010])
])

# define transformation pipeline
transform_train_rotate2 = transforms.Compose([
    # transforms.ToPILImage(),
    # RotationTransform(-30),
    RotationTransform(30),
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                        std=[0.2023, 0.1994, 0.2010])
])

# define transformation pipeline
transform_train_horizontal = transforms.Compose([
    # transforms.ToPILImage(),
    # RotationTransform(-30),
    # RotationTransform(30),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                        std=[0.2023, 0.1994, 0.2010])
])

# define transformation pipeline
transform_train_vertical = transforms.Compose([
    # transforms.ToPILImage(),
    # RotationTransform(-30),
    RotationTransform(30),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                        std=[0.2023, 0.1994, 0.2010])
])

##### Cifar Data
cifar_data = datasets.CIFAR10(root='.',train=True, transform=transform_train, download=True)
    
#We need two copies of this due to weird dataset api 
cifar_data_val = datasets.CIFAR10(root='.',train=True, transform=transform_val, download=True)
    

accs = []

for seed in range(1, 5):
  prng = RandomState(seed)
  random_permute = prng.permutation(np.arange(0, 1000))
  classes =  prng.permutation(np.arange(0,10))
  indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
  indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])


  train_data = Subset(cifar_data, indx_train)
  val_data = Subset(cifar_data_val, indx_val)

  train_data_augment1 = train_data
  train_data_augment2 = train_data
  train_data_augment3 = train_data
  train_data_augment4 = train_data

  train_data_augment1.transform = transform_train_rotate1
  train_data_augment2.transform = transform_train_rotate2
  train_data_augment3.transform = transform_train_horizontal
  train_data_augment4.transform = transform_train_vertical

  combined_dataset = torch.utils.data.ConcatDataset([train_data, train_data_augment1, train_data_augment2, train_data_augment3, train_data_augment4])
  
  combined_dataset_try = torch.utils.data.ConcatDataset([train_data, train_data_augment3])
  # print('Num Samples For Training %d Num Samples For Val %d'%(combined_dataset.indices.shape[0],val_data.indices.shape[0]))
  
  train_loader = torch.utils.data.DataLoader(combined_dataset,
                                             batch_size=64, 
                                             shuffle=True)

  val_loader = torch.utils.data.DataLoader(val_data,
                                           batch_size=64, 
                                           shuffle=False)
  

  model = SmallNet()
  model.to(device)
  epoch=9
  optimizer = torch.optim.SGD(model.parameters(),lr=1e-6, momentum=0.9,
                              weight_decay=5e-4)
  for epoch in range(epoch):
    train(model, device, train_loader, optimizer, epoch, display=epoch%5==0)
    
  accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over 5 instances: %.2f +- %.2f'%(accs.mean(),accs.std()))


Files already downloaded and verified
Files already downloaded and verified
Train Epoch: 0 [174/250 (75%)]	Loss: 2.271920
Train Epoch: 5 [174/250 (75%)]	Loss: 2.275716

Test set: Average loss: 2.2879, Accuracy: 33/400 (8.25%)

Train Epoch: 0 [174/250 (75%)]	Loss: 2.301341
Train Epoch: 5 [174/250 (75%)]	Loss: 2.298358

Test set: Average loss: 2.3027, Accuracy: 21/400 (5.25%)

Train Epoch: 0 [174/250 (75%)]	Loss: 2.274627
Train Epoch: 5 [174/250 (75%)]	Loss: 2.287519

Test set: Average loss: 2.2860, Accuracy: 18/400 (4.50%)

Train Epoch: 0 [174/250 (75%)]	Loss: 2.313755
Train Epoch: 5 [174/250 (75%)]	Loss: 2.311782

Test set: Average loss: 2.3071, Accuracy: 21/400 (5.25%)

Acc over 5 instances: 5.81 +- 1.44
